#  0. Resrobot time table API

In [1]:
from dotenv import load_dotenv
import os
import requests

load_dotenv ()

True

In [14]:
API_KEY = os.getenv("API_KEY")

#from sthlm to gbg
def get_trips(origin_id=740000001, destination_id=740098001):
    """origing_id and destination_id can be found from Stop lookup API"""
    url = f"https://api.resrobot.se/v2.1/trip?format=json&originId={origin_id}&destId={destination_id}&passlist=true&showPassingPoints=true&accessId={API_KEY}"

    

    try:
        response = requests.get(url)
        response.raise_for_status()

        return response.json()
    except requests.exceptions.RequestException as err:
        print(f"Network or HTTP error: {err}")


result = get_trips()
result.keys()

dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])

In [15]:
len(result["Trip"])

5

In [16]:
result["Trip"][0]

{'Origin': {'name': 'Stockholm Centralstation',
  'type': 'ST',
  'id': 'A=1@O=Stockholm Centralstation@X=18058151@Y=59330136@U=1@L=740000001@',
  'extId': '740000001',
  'lon': 18.058151,
  'lat': 59.330136,
  'time': '13:36:00',
  'date': '2025-01-10',
  'minimumChangeDuration': 'PT20M'},
 'Destination': {'name': 'Göteborg Nils Ericsonterminal',
  'type': 'ST',
  'id': 'A=1@O=Göteborg Nils Ericsonterminal@X=11971852@Y=57710271@U=1@L=740020483@',
  'extId': '740020483',
  'lon': 11.971852,
  'lat': 57.710271,
  'routeIdx': 6,
  'prognosisType': 'PROGNOSED',
  'time': '20:00:00',
  'date': '2025-01-10',
  'minimumChangeDuration': 'PT5M'},
 'ServiceDays': [{'planningPeriodBegin': '2025-01-02',
   'planningPeriodEnd': '2025-06-15',
   'sDaysR': 'inte varje dag',
   'sDaysI': '10. jan t o m 11. apr 2025 mån, tor, fre',
   'sDaysB': 'C593264C993264C993264C99300000000000000000'}],
 'LegList': {'Leg': [{'Origin': {'name': 'Stockholm Centralstation',
     'type': 'ST',
     'id': 'A=1@O=Stock

#  b) Find the number of transports arriving to Göteborg centralstationen.

In [31]:
Gbg_Trip = result["Trip"]
Gbg_Trip

[{'Origin': {'name': 'Stockholm Centralstation',
   'type': 'ST',
   'id': 'A=1@O=Stockholm Centralstation@X=18058151@Y=59330136@U=1@L=740000001@',
   'extId': '740000001',
   'lon': 18.058151,
   'lat': 59.330136,
   'time': '13:36:00',
   'date': '2025-01-10',
   'minimumChangeDuration': 'PT20M'},
  'Destination': {'name': 'Göteborg Nils Ericsonterminal',
   'type': 'ST',
   'id': 'A=1@O=Göteborg Nils Ericsonterminal@X=11971852@Y=57710271@U=1@L=740020483@',
   'extId': '740020483',
   'lon': 11.971852,
   'lat': 57.710271,
   'routeIdx': 6,
   'prognosisType': 'PROGNOSED',
   'time': '20:00:00',
   'date': '2025-01-10',
   'minimumChangeDuration': 'PT5M'},
  'ServiceDays': [{'planningPeriodBegin': '2025-01-02',
    'planningPeriodEnd': '2025-06-15',
    'sDaysR': 'inte varje dag',
    'sDaysI': '10. jan t o m 11. apr 2025 mån, tor, fre',
    'sDaysB': 'C593264C993264C993264C99300000000000000000'}],
  'LegList': {'Leg': [{'Origin': {'name': 'Stockholm Centralstation',
      'type': 'S

In [41]:
gbg_arrivals = [
    trip for trip in Gbg_Trip 
    if trip["Destination"]["id"]
]
  
# Display
if gbg_arrivals:
    print(f"Total arrivals at Gothenburg station: {len(gbg_arrivals)}\n")
    for idx, trip in enumerate(gbg_arrivals, start=1):
        destination = trip["Destination"]
        origin = trip["Origin"]
        print(f"Arrival {idx}:")
        print(f"  From: {origin['name']} at {origin['time']}")
        print(f"  To: {destination['name']} at {destination['time']}")
        print("-" * 40)
else:
    print("No arrivals found for Gothenburg station.")


Total arrivals at Gothenburg station: 5

Arrival 1:
  From: Stockholm Centralstation at 13:36:00
  To: Göteborg Nils Ericsonterminal at 20:00:00
----------------------------------------
Arrival 2:
  From: Stockholm Centralstation at 14:07:00
  To: Göteborg Centralstation at 17:41:00
----------------------------------------
Arrival 3:
  From: Stockholm Centralstation at 14:14:00
  To: Göteborg Centralstation at 18:57:00
----------------------------------------
Arrival 4:
  From: Stockholm Centralstation at 14:21:00
  To: Göteborg Nils Ericsonterminal at 21:10:00
----------------------------------------
Arrival 5:
  From: Stockholm Centralstation at 14:28:00
  To: Göteborg Centralstation at 18:00:00
----------------------------------------


#  c) Find the number of transports departuring from Göteborg centralstationen.

In [48]:
API_KEY = os.getenv("API_KEY")
station_id = "740098001"  # Gothenburg Central Station (Gothenburg Nils Ericsonterminal)

# Construct the API URL for departures from Gothenburg
url = f"https://api.resrobot.se/v2.1/departureBoard?id={station_id}&format=json&accessId={API_KEY}"

# Make the API request
response = requests.get(url)
departures = response.json()  # Parse the JSON response

# Check if there are any departures and process the data
if "DepartureBoard" in departures:
    for departure in departures["DepartureBoard"]:
        departure_time = departure.get("scheduledDeparture", "No scheduled departure time")
        train_number = departure.get("trainNumber", "No train number")
        platform = departure.get("platform", "No platform information")
        destination = departure.get("to", {}).get("name", "No destination")
        status = departure.get("status", "No status")

        # Print the departure details
        print(f"Train Number: {train_number}")
        print(f"Destination: {destination}")
        print(f"Scheduled Departure: {departure_time}")
        print(f"Platform: {platform}")
        print(f"Status: {status}")
        print("-" * 40)
else:
    print("No departures data available.")


No departures data available.


In [49]:

API_KEY = os.getenv("API_KEY")
def get_departures(station_id="740098001"):  # Default station ID for Gothenburg
    """Fetch all departures from a given station (Gothenburg in this case)."""
    url = f"https://api.resrobot.se/v2.1/departureBoard?id={station_id}&format=json&accessId={API_KEY}"

    try:
        response = requests.get(url)
        response.raise_for_status()

        return response.json()
    except requests.exceptions.RequestException as err:
        print(f"Network or HTTP error: {err}")
        return None  # Return None if there is an error

# Call the function to get departures
departures_result = get_departures()

# If departures are fetched, print the results
if departures_result:
    for departure in departures_result.get("DepartureBoard", []):
        departure_time = departure.get("scheduledDeparture", "No scheduled departure time")
        train_number = departure.get("trainNumber", "No train number")
        platform = departure.get("platform", "No platform information")
        destination = departure.get("to", {}).get("name", "No destination")
        status = departure.get("status", "No status")

        # Print the departure details
        print(f"Train Number: {train_number}")
        print(f"Destination: {destination}")
        print(f"Scheduled Departure: {departure_time}")
        print(f"Platform: {platform}")
        print(f"Status: {status}")
        print("-" * 40)
    


{'Departure': [{'JourneyDetailRef': {'ref': '1|456|0|1|10012025'},
   'JourneyStatus': 'P',
   'ProductAtStop': {'icon': {'res': 'prod_gen'},
    'operatorInfo': {'name': 'Västtrafik',
     'nameS': '279',
     'nameN': '279',
     'nameL': 'Västtrafik',
     'id': '279'},
    'name': 'Länstrafik - Buss X4',
    'internalName': 'Länstrafik - Buss X4',
    'displayNumber': 'X4',
    'num': 'X4',
    'line': 'X4',
    'lineId': '1279520500001',
    'catOut': 'BLT',
    'catIn': 'BLT',
    'catCode': '7',
    'cls': '128',
    'catOutS': 'BLT',
    'catOutL': 'Länstrafik - Buss',
    'operatorCode': '279',
    'operator': 'Västtrafik',
    'admin': '279___',
    'matchId': 'X4;5205;202'},
   'Product': [{'icon': {'res': 'prod_gen'},
     'operatorInfo': {'name': 'Västtrafik',
      'nameS': '279',
      'nameN': '279',
      'nameL': 'Västtrafik',
      'id': '279'},
     'name': 'Länstrafik - Buss X4',
     'internalName': 'Länstrafik - Buss X4',
     'displayNumber': 'X4',
     'num': '

In [101]:
def get_departures(station_id="740098001"): #Gothenburg
    """Fetch all departures from a given station (Gothenburg in this case)."""
    url = f"https://api.resrobot.se/v2.1/departureBoard?id={station_id}&format=json&accessId={API_KEY}"

    try:
        response = requests.get(url)
        response.raise_for_status()

        return response.json()
    except requests.exceptions.RequestException as err:
        print(f"Network or HTTP error: {err}")
        return None  # Return None if there is an error

# Call the function to get departures
departures_result = get_departures()
departures_result.keys()

dict_keys(['Departure', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId'])

In [102]:
departures_result["Departure"]

[{'JourneyDetailRef': {'ref': '1|15569|0|1|10012025'},
  'JourneyStatus': 'P',
  'ProductAtStop': {'icon': {'res': 'prod_gen'},
   'operatorInfo': {'name': 'Västtrafik',
    'nameS': '279',
    'nameN': '279',
    'nameL': 'Västtrafik',
    'id': '279'},
   'name': 'Länstrafik - Spårväg 7',
   'internalName': 'Länstrafik - Spårväg 7',
   'displayNumber': '7',
   'num': '7',
   'line': '7',
   'lineId': '1279500700001',
   'catOut': 'SLT',
   'catIn': 'SLT',
   'catCode': '6',
   'cls': '64',
   'catOutS': 'SLT',
   'catOutL': 'Länstrafik - Spårväg',
   'operatorCode': '279',
   'operator': 'Västtrafik',
   'admin': '279___',
   'matchId': '7;5007;6569'},
  'Product': [{'icon': {'res': 'prod_gen'},
    'operatorInfo': {'name': 'Västtrafik',
     'nameS': '279',
     'nameN': '279',
     'nameL': 'Västtrafik',
     'id': '279'},
    'name': 'Länstrafik - Spårväg 7',
    'internalName': 'Länstrafik - Spårväg 7',
    'displayNumber': '7',
    'num': '7',
    'line': '7',
    'lineId': '127

In [60]:
len(departures_result["Departure"])

381

In [62]:
departure_count = len(departures_result["Departure"])
departure_count

381

In [67]:
departure_count

381

In [72]:
print(f"Total number of departures from Gothenburg: {departure_count}")


Total number of departures from Gothenburg: 381


In [87]:
departure.keys()

dict_keys(['JourneyDetailRef', 'JourneyStatus', 'ProductAtStop', 'Product', 'Notes', 'name', 'type', 'stop', 'stopid', 'stopExtId', 'lon', 'lat', 'time', 'date', 'reachable', 'direction', 'directionFlag'])

In [78]:
gbg_departures = departures_result.get("Departure", [])

In [109]:
departures.keys()

dict_keys(['Departure', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId'])

In [105]:
print(f"Total number of departures from Gothenburg: {departure_count}")

if gbg_departures:
    for departure in gbg_departures:
                journey_status = departure.get("JourneyStatus", "No status")
                product_name = departure.get("ProductAtStop", {}).get("name", "No product name")
                display_number = departure.get("ProductAtStop", {}).get("displayNumber", "No display number")
                operator = departure.get("ProductAtStop", {}).get("operator", "No operator")
                direction = departure.get("Direction",{}).get("name","No direction")
                
                # Print some departure details
                print(f"Product Name: {product_name}")
                print(f"Display Number: {display_number}")
                print(f"Operator: {operator}")
                print(f"Direction: {direction}")
                print("-" * 40)

Total number of departures from Gothenburg: 381
Product Name: Länstrafik - Buss X4
Display Number: X4
Operator: Västtrafik
Direction: No direction
----------------------------------------
Product Name: Länstrafik - Buss X4
Display Number: X4
Operator: Västtrafik
Direction: No direction
----------------------------------------
Product Name: Länstrafik - Buss 16
Display Number: 16
Operator: Västtrafik
Direction: No direction
----------------------------------------
Product Name: Länstrafik - Buss 17
Display Number: 17
Operator: Västtrafik
Direction: No direction
----------------------------------------
Product Name: Länstrafik - Spårväg 3
Display Number: 3
Operator: Västtrafik
Direction: No direction
----------------------------------------
Product Name: Länstrafik - Spårväg 2
Display Number: 2
Operator: Västtrafik
Direction: No direction
----------------------------------------
Product Name: Länstrafik - Buss SVART
Display Number: SVART
Operator: Västtrafik
Direction: No direction
-----